In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


In [13]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1071c1de0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1071c28f0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [14]:
from langchain_core.messages import HumanMessage

In [15]:
model.invoke([HumanMessage(content="Hi, My name is Aparna")])

AIMessage(content="Hi Aparna!  \n\nIt's nice to meet you. 👋 \n\nIs there anything I can help you with today? \n\n", response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 16, 'total_tokens': 48, 'completion_time': 0.058181818, 'prompt_time': 7.625e-05, 'queue_time': 0.01449284, 'total_time': 0.058258068}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f096e98d-b324-4437-9ff8-5a7cf82f06c3-0', usage_metadata={'input_tokens': 16, 'output_tokens': 32, 'total_tokens': 48})

In [16]:
from langchain_core.messages import AIMessage
model.invoke(
    [
    HumanMessage(content="Hi, my name is aparna and I am a masters student"),
    AIMessage(content="hello !its nice to meet you "),
    HumanMessage(content="Hey wahts my name and waht do I do?")
    ])

AIMessage(content='You told me your name is Aparna and that you are a masters student!  \n', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 52, 'total_tokens': 72, 'completion_time': 0.036363636, 'prompt_time': 0.001833293, 'queue_time': 0.012737677000000001, 'total_time': 0.038196929}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f28f8805-b14b-482e-9df1-a8d07731e0d5-0', usage_metadata={'input_tokens': 52, 'output_tokens': 20, 'total_tokens': 72})

### Message History

We can use a Message History class to wrap our model and make it stateful.This will keep track of inputs and outputs of the model, and store them in some datastore.

In [17]:
!pip install langchain_community

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [21]:
config={"configurable":{"session_id":"chat1"}}

In [23]:
response=with_message_history.invoke([
    HumanMessage(content="Hi, I am aparna and I am a masters student")],
    config=config
)
response.content

"Hi Aparna! It's nice to meet you.  \n\nWhat subject are you studying for your master's degree?  \n\nI'm curious to know more about your field of study! 😊 \n"

In [25]:
with_message_history.invoke([
    HumanMessage(content="Hi, what is my name ")],
    config=config
)


AIMessage(content='Your name is Aparna.  😊  I remember!  \n\nDo you have any other questions for me?  \n', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 190, 'total_tokens': 217, 'completion_time': 0.049090909, 'prompt_time': 0.005668718, 'queue_time': 0.008663612000000001, 'total_time': 0.054759627}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-139228ee-3d80-4760-8d4c-15905267ae39-0', usage_metadata={'input_tokens': 190, 'output_tokens': 27, 'total_tokens': 217})

In [28]:
## change the config-->session id

config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [
        HumanMessage(content="whats my name")],
        config=config1
    
)
response.content

"As an AI, I don't have access to any personal information about you, including your name.\n\nIf you'd like to tell me your name, I'm happy to know! 😊  \n\n"

### Prompt templates

Prompt templates helps to turn raw user infromation into a format that the LLM work with.In this case the raw user input is just a message, which we are passing to the LLM.

In [36]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate(
    [
        ("system","Your are a helpful assistant.Answer all the questions to the best of your ability in {language}",),
        
        MessagesPlaceholder(variable_name="messages"),

    ]
)
chain= prompt | model

In [37]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is aparna")],
                       "language":"Hindi"})

In [40]:
chain.invoke({"messages":[HumanMessage(content="Hi, My name is aparna")],"language":"Hindi"})

AIMessage(content='नमस्ते Aparna! बहुत अच्छा है आपने मुझे ढूंढ लिया। \n\nआप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा आपको सही जवाब देना।  \n\n', response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 34, 'total_tokens': 88, 'completion_time': 0.098181818, 'prompt_time': 0.000388039, 'queue_time': 0.013058271, 'total_time': 0.098569857}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c0779136-44cb-4049-b03f-a31e88a49ebc-0', usage_metadata={'input_tokens': 34, 'output_tokens': 54, 'total_tokens': 88})

In [32]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [33]:
config ={"configurable": {"session_id":"chtat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="hi My name is aparna")],
    config=config
)
response

AIMessage(content="Hi Aparna! \n\nIt's nice to meet you. 😊  \n\nHow can I help you today?  \n\n", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 31, 'total_tokens': 60, 'completion_time': 0.052727273, 'prompt_time': 0.000382518, 'queue_time': 0.014473989999999999, 'total_time': 0.053109791}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1117fbd6-6025-4a84-a0d0-87f27c29264b-0', usage_metadata={'input_tokens': 31, 'output_tokens': 29, 'total_tokens': 60})

In [41]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [42]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi I am aparna")],"language":"Hindi"},
    config=config

)
response.content

'नमस्ते Aparna!  😊  मुझे बहुत अच्छा लग रहा है आपसे बातचीत करने का।  \n\nआप मुझसे क्या पूछना चाहती हैं? \n'

### Managing the conversation History


In [43]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=70, 
    strategy="last",
    token_coutner=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)